In [51]:
import os

import numpy as np

In [52]:
# Get the path to the Downloads folder
downloads_folder = os.path.expanduser("~" + os.sep + "Downloads")
print("path: " + downloads_folder)

# List the files and directories in the Downloads folder
downloads_contents = os.listdir(downloads_folder)

# Print the list of files and directories
for item in downloads_contents:
    print(item)


path: /Users/chrismaltez/Downloads
202012_Tax Return Transcript_MALT_102002725519.pdf
borrow.mp3
connect-four-oo
Sync Manager.png
MI-005_Miami (2).pdf
StoneMarbleCalacatta004
cadsUpload.PNG
bootstrap-5.3.0-alpha1 (1).zip
Discord.dmg
01-starting-project 2
avi-werde-hHz4yrvxwlA-unsplash.jpg
Revit Monkey.rtfd
Votive2022 (1).vsix
react-state-carousel-coins-solution.zip
routing--react-suspense-start.zip
Export+BIM+360+Projects.zip
SpearFishingReservation-LaJollaWatersports_SD.pdf
RainbowBraces.vsix
TerrazzoVenetianMatteWhite001.zip
Maltez_Christopher_Resume.pdf
IMG_2129.HEIC
kitchen_wood_1k.blend
connect-four-oo 3
Thumbs.db
2022_San_Pablo_Recreation_Use_Fees.pdf
101280971-cross-icon-in-circle-can-be-used-as-delete-block-close-button-etc-delete-x-cross-rounded-icon-is-fla.webp
HordeSymbol.ai
Kaiser Platinum 90 HMO 2 20 (EOC) 120122.pdf
IMG_2393.mp4
2021 - Chris Maltez Tax Returns.pdf
CHRISTOPHER_MALTEZ_RESUME_October_2018 (1).pages
snippets
CADExporter.sln
Marble13
IMG_2294.HEIC
202103260929

In [53]:
# Specify the file name you want to retrieve
# file_name = "sample_brake_event.csv"  # Replace with the name of your file
file_name = "challenge1_data.csv"  # Replace with the name of your file

# Construct the source path (Downloads folder)
downloads_folder = os.path.expanduser("~" + os.sep + "Downloads")
source_path = os.path.join(downloads_folder, file_name)

# Check if the file exists in the Downloads folder
if os.path.exists(source_path):
    print(f"Found csv: {file_name}")
else:
    print(f"did not find file")

Found csv: challenge1_data.csv


In [54]:
# !pip3 install pandas
import pandas as pd

pd.set_option('display.float_format', lambda x: '{:.0f}'.format(x))


print(f"source path: {source_path}")
df = pd.read_csv(source_path)

source path: /Users/chrismaltez/Downloads/challenge1_data.csv


In [56]:
# Create a DataFrame from the sample data
df = pd.DataFrame(df)

# Apply one-hot encoding to the 'step' and 'braking_id' columns
df_encoded = pd.get_dummies(df, columns=['compound_id', 'step', 'braking_id'], dtype=int)

# Print the resulting DataFrame with one-hot encoding
print(df_encoded.head())

   time_index  time  pressure_bar  temp_c  speed_kph  mu  A_1  C_1  B_1  C_2  \
0           1     1             9      27         73   1    0    5    0    0   
1           2     1             9      28         72   1    0    5    0    0   
2           3     1             9      28         71   1    0    5    0    0   
3           4     1             9      29         70   1    0    5    0    0   
4           5     1             8      29         68   1    0    5    0    0   

   ...  braking_id_fading_2_8  braking_id_fading_2_9  \
0  ...                      0                      0   
1  ...                      0                      0   
2  ...                      0                      0   
3  ...                      0                      0   
4  ...                      0                      0   

   braking_id_increasing_temperature_1  braking_id_increasing_temperature_2  \
0                                    0                                    0   
1                       

In [57]:
# Assuming 'df' is your DataFrame
column_headers = df_encoded.columns

In [58]:
# !pip install matplotlib
X = df_encoded.drop(columns=column_headers)  # Include all columns except 'pressure_bar' and 'temp_c'
y = df_encoded[column_headers]  

In [59]:

# Split the dataset into training, validation, and test sets
X_train, y_train = X[:60000], y[:60000]
X_val, y_val = X[60000:65000], y[60000:65000]
X_test, y_test = X[65000:], y[65000:]

In [60]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

In [61]:
# Build a multi-output model
model = Sequential()
model.add(InputLayer((X.shape[1],)))
model.add(Dense(64, 'relu'))
model.add(Dense(545, 'linear'))  # Two output layers for 'pressure_bar' and 'temp_c'


In [64]:
# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

In [65]:
# Train the model
cp1 = ModelCheckpoint('model1/', save_best_only=True)
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=1, callbacks=[cp1])
# model.fit(X_train1, y_train1, validation_data=(X_val1, y_val1), epochs=10, callbacks=[cp1])


1866/1875 [============================>.] - ETA: 0s - loss: 8242.8838INFO:tensorflow:Assets written to: model1/assets


INFO:tensorflow:Assets written to: model1/assets


1875/1875 [==============================] - 7s 4ms/step - loss: 8244.9590 - val_loss: 9953.2910


In [66]:
# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
from tensorflow.keras.models import load_model

# Load the trained model
loaded_model = load_model('model1/')  # Replace with the correct path to your saved model

38451/38451 [==============================] - 89s 2ms/step - loss: 8449.4561
Test Loss: 8449.4560546875


In [67]:
# Generate predictions for 'X_new'
predictions = loaded_model.predict(X_test)

# 'predictions' will contain the predicted values for both 'pressure_bar' and 'temp_c'


38451/38451 [==============================] - 88s 2ms/step


In [68]:
# Convert the predictions array into a DataFrame with appropriate column names
predictions_df = pd.DataFrame(predictions, columns=column_headers)

# Now, 'predictions_df' contains the predicted values for 'pressure_bar' and 'temp_c'
print(predictions_df.head())

   time_index  time  pressure_bar  temp_c  speed_kph  mu  A_1  C_1  B_1  C_2  \
0         102     4             9     102         60   1    1    3    0    0   
1         102     4             9     102         60   1    1    3    0    0   
2         102     4             9     102         60   1    1    3    0    0   
3         102     4             9     102         60   1    1    3    0    0   
4         102     4             9     102         60   1    1    3    0    0   

   ...  braking_id_fading_2_8  braking_id_fading_2_9  \
0  ...                      0                      0   
1  ...                      0                      0   
2  ...                      0                      0   
3  ...                      0                      0   
4  ...                      0                      0   

   braking_id_increasing_temperature_1  braking_id_increasing_temperature_2  \
0                                    0                                    0   
1                       

In [69]:
my_csv = predictions_df.head(1)
print(predictions_df.head(1))

   time_index  time  pressure_bar  temp_c  speed_kph  mu  A_1  C_1  B_1  C_2  \
0         102     4             9     102         60   1    1    3    0    0   

   ...  braking_id_fading_2_8  braking_id_fading_2_9  \
0  ...                      0                      0   

   braking_id_increasing_temperature_1  braking_id_increasing_temperature_2  \
0                                    0                                    0   

   braking_id_increasing_temperature_3  braking_id_increasing_temperature_4  \
0                                    0                                    0   

   braking_id_increasing_temperature_5  braking_id_increasing_temperature_6  \
0                                    0                                    0   

   braking_id_increasing_temperature_7  braking_id_increasing_temperature_8  
0                                    0                                    0  

[1 rows x 545 columns]


In [74]:
column_headers = predictions_df.columns
for column in column_headers:
    print(column)

time_index
time
pressure_bar
temp_c
speed_kph
mu
A_1
C_1
B_1
C_2
F_1
B_2
E_1
B_3
B_4
F_2
F_3
B_5
C_3
F_4
F_5
B_6
B_7
B_8
A_2
A_3
B_9
B_10
C_4
B_11
B_12
E_2
E_3
E_4
C_5
B_13
F_6
C_6
B_14
B_15
E_5
B_16
A_4
F_7
F_8
E_6
E_7
B_17
B_18
B_19
E_8
F_9
B_20
F_10
B_21
A_5
E_9
F_11
F_12
E_10
B_22
D_1
E_11
B_23
B_24
A_6
compound_id_17937-100
compound_id_17937-101
compound_id_17937-102
compound_id_17937-103
compound_id_17937-104
compound_id_17937-105
compound_id_17937-106
compound_id_17937-107
compound_id_17937-108
compound_id_17937-109
compound_id_17937-110
compound_id_17937-111
compound_id_17937-112
compound_id_17937-113
compound_id_17937-114
compound_id_17937-115
compound_id_17937-116
compound_id_17937-117
compound_id_17937-120
compound_id_17937-121
compound_id_17937-130
compound_id_17937-131
compound_id_17937-132
compound_id_17937-133
compound_id_17937-134
compound_id_17937-135
compound_id_17937-136
compound_id_17937-137
compound_id_17937-138
compound_id_17937-139
compound_id_17937-140
compound_

In [76]:
# compound_id_columns = [column.startswith("compound_id") if column in column_headers]
# TODO: LEFT OFF HERE

In [77]:
predictions_df.head().to_csv('my_data.csv', index=False)



In [78]:
# Read the CSV file
df_saved = pd.read_csv('my_data.csv')


# Display the DataFrame
df_saved.head()  # Use .head() to display the first few rows


,time_index,time,pressure_bar,temp_c,speed_kph,mu,A_1,C_1,B_1,C_2,...,braking_id_fading_2_8,braking_id_fading_2_9,braking_id_increasing_temperature_1,braking_id_increasing_temperature_2,braking_id_increasing_temperature_3,braking_id_increasing_temperature_4,braking_id_increasing_temperature_5,braking_id_increasing_temperature_6,braking_id_increasing_temperature_7,braking_id_increasing_temperature_8
0,102,4,9,102,60,1,1,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,102,4,9,102,60,1,1,3,0,0,...,0,0,0,0,0,0,0,0,0,0
2,102,4,9,102,60,1,1,3,0,0,...,0,0,0,0,0,0,0,0,0,0
3,102,4,9,102,60,1,1,3,0,0,...,0,0,0,0,0,0,0,0,0,0
4,102,4,9,102,60,1,1,3,0,0,...,0,0,0,0,0,0,0,0,0,0
